<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/China_A_share_market_tushare.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [ ]:
#colab:https://colab.research.google.com/github/yfxc/my_tutorials/blob/master/1-Introduction/China_A_share_market_tushare.ipynb
#!git clone https://github.com/AI4Finance-Foundation/FinRL-Meta

In [2]:
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.8 MB/s eta 0:00:00
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package swig4.0.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting

In [ ]:
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
!conda config --set channel_priority strict
!conda install -c conda-forge ta-lib

Install other libraries

In [ ]:
!pip install stockstats
!pip install tushare
#install talib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib')
!./configure --prefix=/usr
!make
!make install
#!sudo make install # Sometimes it need root
os.chdir('../')

## Import Modules

In [1]:
%cd ../FinRL-Meta
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
from IPython import display

#display.set_matplotlib_formats("svg")

from meta import config
from meta.data_processor import DataProcessor
from main import check_and_make_directories
from meta.data_processors.tushare import Tushare, ReturnPlotter
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv
from agents.stablebaselines3_models import DRLAgent
import os
from typing import List
from argparse import ArgumentParser
from meta import config
from meta.config_tickers import DOW_30_TICKER
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

/home/gez-pc/git/my_tutorials/FinRL-Meta
ALL Modules have been imported!


## Create Folders

In [2]:
import os

'''
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"):
  os.makedirs("./datasets")
if not os.path.exists("./trained_models"):
  os.makedirs("./trained_models")
if not os.path.exists("./tensorboard_log"):
  os.makedirs("./tensorboard_log")
if not os.path.exists("./results"):
  os.makedirs("./results")
'''

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

## Download data, cleaning and feature engineering

In [4]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01'
TRAIN_END_DATE= '2019-08-01'
TRADE_START_DATE = '2019-08-01'
TRADE_END_DATE = '2020-01-03'

TIME_INTERVAL = "1d"
kwargs = {}
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

tushare is NOT supported yet.


ValueError: Please input correct account info for tushare!

### Download and Clean

In [3]:
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

NameError: name 'p' is not defined

### Add technical indicator

In [6]:
p.add_technical_indicator(config.INDICATORS)
p.fillna()

#print(f"p.dataframe: {p.dataframe}")

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


## Split training dataset

In [7]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE)

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [8]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']


In [9]:
print(f"train.head(): {train.head()}")

train.head():          tic        time  index   open   high    low  close  adjusted_close  \
0  600000.SH  2015-01-08     45  15.87  15.88  15.20  15.25           15.25   
0  600009.SH  2015-01-08     46  20.18  20.18  19.73  20.00           20.00   
0  600016.SH  2015-01-08     47  10.61  10.66  10.09  10.20           10.20   
0  600028.SH  2015-01-08     48   7.09   7.41   6.83   6.85            6.85   
0  600030.SH  2015-01-08     49  36.40  36.70  34.68  35.25           35.25   

       volume      macd    boll_ub    boll_lb     rsi_30      cci_30  \
0  3306271.72 -0.032571  16.617911  15.012089   6.058641 -125.593009   
0   198117.45 -0.016008  20.663897  19.736103  12.828915  -90.842491   
0  4851684.17 -0.018247  10.957604   9.997396  11.862558  -99.887006   
0  8190902.35 -0.008227   7.342000   6.743000  27.409248   36.578171   
0  6376268.69  0.032910  36.576444  33.808556  61.517448   47.947020   

        dx_30  close_30_sma  close_60_sma  
0   23.014040       15.8150       

In [10]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [11]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


## Train

In [12]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [13]:
env_train, _ = e_train_gym.get_sb_env()

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


### DDPG

In [14]:
agent = DRLAgent(env=env_train)
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", }
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300]))
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cuda device
Logging to tensorboard_log/ddpg/ddpg_6
Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1204246.50
total_reward: 204246.50
total_cost: 12413.50
total_trades: 1260
Sharpe: 0.292
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 1531759.03
total_reward: 531759.03
total_cost: 630.97
total_trades: 81
Sharpe: 0.506
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 2802990.35
total_reward: 1802990.35
total_cost: 180.65
total_trades: 36
Sharpe: 0.977
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 2814953.29
total_reward: 1814953.29
total_cost: 180.71
total_trades: 38
Sharpe: 0.991
----------------------------------
| time/             

### A2C

In [ ]:
agent = DRLAgent(env=env_train)
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_1
--------------------------------------
| time/                 |            |
|    fps                | 251        |
|    iterations         | 100        |
|    time_elapsed       | 1          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -21.3      |
|    explained_variance | -0.0322    |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -2.66      |
|    reward             | -0.5146969 |
|    std                | 1          |
|    value_loss         | 2.24       |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 248       |
|    iterations         | 200       |
|    time_elapsed       | 4         |
|    total_timesteps    | 1000      |
| train/                |           

## Trade

In [17]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE)
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True }
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

KeyboardInterrupt: 

In [16]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1206230.32
total_reward: 206230.32
total_cost: 68.68
total_trades: 5
Sharpe: 2.046


KeyError: 'date'

In [ ]:
df_actions.to_csv("action.csv", index=False)
print(f"df_actions: {df_actions}")

df_actions:             600000.SH  600009.SH  600016.SH  600028.SH  600030.SH  600031.SH  \
date                                                                           
2019-08-01          0          0       1000       1000          0       1000   
2019-08-02          0          0       1000       1000          0       1000   
2019-08-05          0          0       1000       1000          0       1000   
2019-08-06          0          0       1000       1000          0       1000   
2019-08-07          0          0       1000       1000          0       1000   
...               ...        ...        ...        ...        ...        ...   
2019-12-25          0          0          0          0          0          0   
2019-12-26          0          0          0          0          0          0   
2019-12-27          0          0          0          0          0          0   
2019-12-30          0          0          0          0          0          0   
2019-12-31          0       

## Backtest

### matplotlib inline

In [ ]:
plotter = ReturnPlotter(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot()

In [ ]:
# ticket: SSE 50：000016
plotter.plot("000016")

### CSI 300

In [ ]:
baseline_df = plotter.get_baseline("399300")

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")